# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to Moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model
**Q1**: Expand the above definition of $ p(\vec{w})$ using naive estimates of the parameters, such as $  p(w_4 \mid w_2, w_3) \stackrel{\tiny{\mbox{def}}}{=}  \frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} $ where \( C(w_2 w_3 w_4) \) denotes the count of times the trigram $ w_2 w_3 w_4 $ was observed in a training corpus.

**Write your answer:**

\# todo



**Q2**: One could also define a kind of reversed trigram language model $p_{reversed}$ that instead assumed the words were generated in reverse order (from right to left):
\begin{align} p_{reversed}(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}&p(w_n) \cdot p(w_{n-1} \mid w_n) \cdot p(w_{n-2} \mid w_{n-1} w_n) \cdot p(w_{n-3} \mid w_{n-2} w_{n-1}) \\ &\cdots p(w_2 \mid w_3 w_4) \cdot p(w_1 \mid w_2 w_3) \end{align}
By manipulating the notation, show that the two models are identical, i.e., $ p(\vec{w}) = p_{reversed}(\vec{w}) $ for any $ \vec{w} $ provided that both models use MLE parameters estimated from the same training data (see Q1 above).

**Write your answer:**

\# todo



## 2 $N$-gram Language Model Implementation

In [ ]:
!wget -O train.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/test.txt

### 2.1 Building vocabulary

**Code**

\# todo



In [ ]:
from collections import Counter
import re
import numpy as np
from functools import reduce
from collections import defaultdict

In [ ]:
dataset = 'train'

tokenized_data = []
tokenized_data_2dim = []
with open(f'data/lm/{dataset}.txt', encoding='utf-8') as fd:
    lines = fd.readlines()
    for line in lines:
        words = ['<s>'] + re.split(r'\s+', line.strip()) + ['</s>']
        tokenized_data.extend(words)
        tokenized_data_2dim.append(words)
        
# Step 1: Count token frequencies
token_counts = Counter(tokenized_data)

# Step 2: Replace infrequent tokens with "<UNK>"
min_token_freq = 3
tokenized_data_with_unk = [token if token_counts[token] >= min_token_freq else '<UNK>' for token in tokenized_data]
tokenized_data_with_unk_2dim = [[token if token_counts[token] >= min_token_freq else '<UNK>' for token in line] for line in tokenized_data_2dim]

# Step 3: Add start-of-sentence and end-of-sentence tokens
vocabulary = list(set(tokenized_data_with_unk))
vocabulary_size = len(vocabulary)

print("Vocabulary size:", vocabulary_size)
# print('<UNK>' in vocabulary)
# print(len(token_counts))

In [ ]:
# # for bigram
# from collections import defaultdict
# biCnt = defaultdict(lambda: defaultdict(int))

# # To calculate the occurrence count of wi-1 wi
# with open(f'data/lm/{dataset}.txt', encoding='utf-8') as fd:
#     lines = fd.readlines()
#     for line in lines:
#         words = re.split(r'\s+', line.strip())
#         for i in range(1, len(words)):
#             if words[i - 1] in unk:
#                 words[i - 1] = '<UNK>'
#             if words[i] in unk:
#                 words[i] = '<UNK>'
#             if words[i] == '':
#                 print(words)
#             biCnt[words[i - 1]][words[i]] += 1
#         biCnt['<s>'][words[0]] += 1
#         biCnt[words[-1]]['</s>'] += 1
        
# print(len(biCnt))
# # assert len(biCnt) == len(uniCnt) - 1
# # assert sum(biCnt['<s>'].values()) == uniCnt['<s>']
# print(biCnt['<s>'])

**Discussion**

unigram:

the number of parameters: Vocabulary size 19349

bigram:

the number of parameters: Vocabulary size^2 19349**2

trigram:

the number of parameters: Vocabulary size^3 19349**3



### 2.2 $N$-gram Language Modeling

**Code**

\# todo



In [ ]:
"""
unigram
"""
unigram_counts = Counter(tokenized_data_with_unk)

def calculate_unigram_prob(word):
    return unigram_counts[word] / len(tokenized_data_with_unk)

# Step 4: Calculate perplexity
def calculate_perplexity(data):
    num_words = len(data)
    log_prob_sum = 0

    for i in range(num_words):
        word = data[i]
        prob = calculate_unigram_prob(word)
        log_prob_sum += np.log(prob)

    perplexity = np.exp(log_prob_sum) ** (-1/num_words)
    return perplexity

# Calculate perplexity for each sentence
sentence_perplexities = []
for i, sentence in enumerate(tokenized_data_with_unk_2dim):
    perplexity = calculate_perplexity(sentence)
    sentence_perplexities.append(perplexity)


print("Perplexity for each sentence:")
for i, perplexity in enumerate(sentence_perplexities):
    print(f"Sentence {i+1}: {perplexity}")


In [ ]:
# # unigram
# class Unigram():
#     def train(self, data):
#         pass

# # calc prob for unigram
# prob_d = {}
# for word, cnt in voc.items():
#     prob_d[word] = cnt / word_cnt
# print(prob_d['<s>'])
# print(prob_d['unk'])

# # prob for all
# # perp = 1
# # for prob in prob_d.values():
# #     perp *= prob ** (-1/len(prob_d))

# # Calculate the perp of each sentence and then calculate the mean
# perps = []
# with open('data/lm/dev.txt', encoding='utf-8') as fd:
#     lines = fd.readlines()
#     for line in lines:
#         words = line.split(' ')
#         perp = 1
#         for word in words:
#             if word in prob_d:
#                 perp *= prob_d[word] ** (-1/len(words))
#             else:
#                 perp *= prob_d['unk'] ** (-1/len(words))
#         # perp = perp ** (-1/len(words))
#         perps.append(perp)
#         # print(perp)
# minPerp = sum(perps) / len(perps)
# minPerp

In [ ]:
"""
bigram
"""
# Step 1: Count bigram frequencies for each sentence
bigram_counts_list = [Counter(zip(sentence, sentence[1:])) for sentence in tokenized_data_with_unk_2dim]
bigram_counts = Counter()
for count in bigram_counts_list:
    bigram_counts.update(count)

# Step 3: Calculate add-one smoothed probabilities for each sentence
# vocab_sizes = [len(unigram_count) for unigram_count in unigram_counts]
# total_bigrams = [sum(bigram_count.values()) for bigram_count in bigram_counts]

def calculate_bigram_prob(word1, word2):
    return bigram_counts[(word1, word2)] / unigram_counts[word1]

# Step 4: Calculate perplexity for each sentence
def calculate_perplexity(sentence):
    num_words = len(sentence)
    log_prob_sum = 0

    for i in range(1, num_words):
        word1 = sentence[i - 1]
        word2 = sentence[i]
        prob = calculate_bigram_prob(word1, word2)
        log_prob_sum += np.log(prob)

    perplexity = np.exp(log_prob_sum) ** (-1/num_words)
    return perplexity

# Calculate perplexity for each sentence
sentence_perplexities = []
for i, sentence in enumerate(tokenized_data_with_unk_2dim):
    perplexity = calculate_perplexity(sentence)
    sentence_perplexities.append(perplexity)

print("Perplexity for each sentence:")
for i, perplexity in enumerate(sentence_perplexities):
    print(f"Sentence {i+1}: {perplexity}")

In [ ]:
# # bigram

# from collections import defaultdict
# # calc conditional perp
# biProbs = defaultdict(lambda: defaultdict(int))
# for wi in biCnt:
#     wiCnt = 0
#     for _, cnt in biCnt[wi].items():
#         wiCnt += cnt
#     for wj, cnt in biCnt[wi].items():
#         biProbs[wi][wj] = cnt / wiCnt
# print(biProbs['<s>'])
# print(biProbs['The'])
# assert sum(biProbs['<s>'].values()) - 1 < 1e-5
# assert sum(biProbs['The'].values()) - 1 < 1e-5

# # Calculate the perp of each sentence and then calculate the mean
# def calc_perp(biProbs):
#     perps = []
#     with open(f'data/lm/{dataset}.txt', encoding='utf-8') as fd:
#         lines = fd.readlines()
#         for line in lines:
#             words = line.split(' ')
#             perp = biProbs['<s>'][words[0]] ** (-1/len(words))
#             for i in range(1, len(words)):
#                 if biProbs[words[i - 1]][words[i]] != 0:
#                     perp *= biProbs[words[i - 1]][words[i]] ** (-1/len(words))
                
#             perp *= biProbs[words[-1]]['</s>'] ** (-1/len(words))
#             # perp = perp ** (-1/len(words))
#             # print(perp)
#             perps.append(perp)
#     return perps
    
# perps = calc_perp(biProbs)
# print(perps[:10])

**Discussion**

The lower the Perplexity value, the more accurate the model's prediction of the language sequence.

Under the same corpus and model training conditions, the perplexity of the unigram model is higher and the perplexity of the bigram model is lower. Thus, the prediction of the bigram model is more accurate. This is because the bigram model takes into account neighboring The contextual information between words can capture local semantic and syntactic relationships better than the unigram model.



### 2.3 Smoothing

#### 2.3.1 Add-one (Laplace) smoothing

**Code**

\# todo



In [ ]:
# from copy import deepcopy
# def getOovLaplace():
#     oovLaplace = deepcopy(biCnt)
#     # 计算新的条件概率
#     biLaplaceProbs = defaultdict(lambda: defaultdict(int))
#     for wi in oovLaplace:
#         # add one
#         for wj in oovLaplace:
#             oovLaplace[wi][wj] += 1
#         wiCnt = 0
#         for wj, cnt in oovLaplace[wi].items():
#             wiCnt += cnt
#         for wj, cnt in oovLaplace[wi].items():
#             biLaplaceProbs[wi][wj] = cnt / wiCnt
#         print(oovLaplace[wi])
#         return
# biLaplaceProbs = getOovLaplace()

# # 计算新的perplexity
# perps = calc_perp(biLaplaceProbs)
# print(perps[:10])

In [ ]:
def calculate_add_one_smoothed_prob(word1, word2):
    numerator = bigram_counts[(word1, word2)] + 1
    denominator = unigram_counts[word1] + vocabulary_size
    return numerator / denominator


class Bigram():
    def __init__(self):
        self.prob_list = defaultdict(lambda: defaultdict(int))

    def train(self):
        for sentence in tokenized_data_with_unk_2dim:
            num_words = len(sentence)

            for i in range(1, num_words):
                word1 = sentence[i - 1]
                word2 = sentence[i]
                prob = calculate_add_one_smoothed_prob(word1, word2)
                self.prob_list[word1][word2] = prob

    def get(self, word1, word2):
        return self.prob_list[word1][word2]

    def predict(self):
        pass


bigram = Bigram()
bigram.train()

perplexity_train = []
for sentence in tokenized_data_with_unk_2dim:
    num_words = len(sentence)
    log_prob_sum = 0

    for i in range(1, num_words):
        word1 = sentence[i - 1]
        word2 = sentence[i]
        perp = np.log(bigram.get(word1, word2))
        log_prob_sum += perp

    perplexity = np.exp(log_prob_sum) ** (-1/num_words)
    perplexity_train.append(perplexity)

print("Perplexity for each sentence:")
for i, perplexity in enumerate(perplexity_train):
    print(f"Sentence {i+1}: {perplexity}")

**Discussion**

perplexity increases



#### 2.3.2: Add-$k$ smoothing

**Code**

\# todo



**Discussion**

\# todo



#### 2.3.3 Linear Interpolation

In [ ]:
"""
trigram
"""
from collections import Counter

tokenized_data = ['I', 'like', 'apples', 'and', 'bananas', 'I', 'like', 'apples']
trigram_counts = Counter(zip(tokenized_data, tokenized_data[1:], tokenized_data[2:]))
print(trigram_counts)

In [ ]:
lambda1 = 0.2  # Weight for unigram
lambda2 = 0.4  # Weight for bigram
lambda3 = 0.4  # Weight for trigram

def linear_interpolation_smoothing(word1, word2, word3):
    # Calculate the interpolated probability using weighted averages
    unigram_prob = unigram_model.get(word3, 0.0)
    bigram_prob = bigram.get((word2, word3), 0.0)
    trigram_prob = trigram_model.get((word1, word2, word3), 0.0)

    interpolated_prob = lambda1 * unigram_prob + lambda2 * bigram_prob + lambda3 * trigram_prob
    return interpolated_prob

**Code**

\# todo



**Discussion**

\# todo



##### **Optimization**:

\# todo

## 3 Preposition Prediction

In [ ]:
!wget -O dev.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
!wget -O dev.out https://github.com/qtli/COMP7607-Fall2023/blob/master/assignments/A1/data/prep/dev.out

In [31]:
"""
read dev.in & prep by bigram
"""
prep_file = []
with open(f'data/prep/dev.in', encoding='utf-8') as fd:
    with open('data/prep/prep.out', 'w', encoding='utf-8') as file:
        lines = fd.readlines()
        for line in lines:
            words = line.split(' ')
            
            prep_line = []
            for i in range(len(words)):
                if (words[i] == '<PREP>'):
                    dic = bigram.prob_list[words[i - 1]]
                    # cope with UNK
                    if words[i - 1] not in vocabulary or not dic:
                        dic = bigram.prob_list["<UNK>"]

                    # prep by bigram
                    prep = max(dic, key=dic.get)
                    prep_line.append(prep)
            
            prep_file.append(" ".join(prep_line))

            file.write(" ".join(prep_line) + '\n')

        

In [32]:
"""
calculate the accuracy of the bigram
"""
total = 0
right = 0
with open(f'data/prep/dev.out', encoding='utf-8') as fd_dev:
    with open('data/prep/prep.out', encoding='utf-8') as fd_prep:
        lines_dev = fd_dev.readlines()
        lines_prep = fd_prep.readlines()
        for i in range(len(lines_dev)):
            line_dev = lines_dev[i].strip().split()
            line_prep = lines_prep[i].strip().split()

            for j in range(len(line_dev)):
                right += line_dev[j] == line_prep[j]
            total += len(lines_dev)
print(right, total)

864 1327104
